In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict

c:\Python\Python396\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
torch.set_grad_enabled(False)

In [3]:
def get_x(shape= (4,3,32,224,224)):
    x_pt = torch.rand(shape) * 255
    x_np = x_pt.numpy()
    x_tf = tf.convert_to_tensor(x_np)

    return x_tf, x_pt

In [4]:
shape_of_input = [10,3,32,224,224]
x_tf, x_pt = get_x(shape_of_input)

## Convert Weights

Initiate model and Load PyTorch Weights

In [5]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
del cfg["drop_path_rate"]
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

0

In [6]:
print("shape_of_input", shape_of_input)
print("pt")
pt_model = SwinTransformer3D_pt(**cfg,drop_rate=0.4, drop_path_rate=0., isTest= True)
print("++++++++\n tf")
tf_model = SwinTransformer3D(**cfg,shape_of_input=shape_of_input, drop_rate=0.4, drop_path_rate=0., isTest= True)
x_tf, x_pt = get_x()


print("\n\n\n pt")
basic_pt, z= pt_model(x_pt)
print("------- \n tf")

basic_tf, y = tf_model(x_tf)

# print("\n loading checkpoint")
# checkpoint = torch.load(f'{name}.pth')
# new_state_dict = OrderedDict()
# for k, v in checkpoint['state_dict'].items():
#     if 'backbone' in k:
#         name = k[9:]
#         new_state_dict[name] = v 

# pt_model.load_state_dict(new_state_dict) 
# pt_model.eval()

shape_of_input [10, 3, 32, 224, 224]
pt


c:\Python\Python396\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


++++++++
 tf
shape_of_input:  [10, 3, 32, 224, 224]



 pt
------- 
 tf


In [ ]:
# a = a

In [ ]:
pt_model.eval()


Convert Functions

In [ ]:
def conv_transpose(w):
    return w.transpose(2,3,4,1, 0)
    

def modify_tf_block( tf_component, pt_weight,  pt_bias = None, is_attn=False):
    in_shape = pt_weight.shape

    if isinstance(tf_component, tf.keras.layers.Conv3D) :
      pt_weight = conv_transpose(pt_weight)

    if isinstance(tf_component, tf.keras.layers.Dense) and not is_attn:
      pt_weight =pt_weight.transpose()

    if isinstance(tf_component, (tf.keras.layers.Dense, tf.keras.layers.Conv3D)):
        tf_component.kernel.assign(tf.Variable(pt_weight))

        if pt_bias is not None:
            tf_component.bias.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, tf.keras.layers.LayerNormalization):

        tf_component.gamma.assign(tf.Variable(pt_weight))

        tf_component.beta.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, (tf.Variable)):
        tf_component.assign(tf.Variable(pt_weight))

    else:
        return tf.convert_to_tensor(pt_weight)
        
        

    return tf_component


def modify_swin_blocks(np_state_dict, pt_weights_prefix, tf_block):

  for layer in tf_block:
    if isinstance(layer, PatchMerging):
      patch_merging_idx = f"{pt_weights_prefix}.downsample"

      layer.reduction = modify_tf_block( layer.reduction,
                          np_state_dict[f"{patch_merging_idx}.reduction.weight"])
      layer.norm = modify_tf_block( layer.norm,
                        np_state_dict[f"{patch_merging_idx}.norm.weight"],
                        np_state_dict[f"{patch_merging_idx}.norm.bias"]
                        )
      
  # Swin Layers
  common_prefix = f"{pt_weights_prefix}.blocks"
  block_idx = 0

  for outer_layer in tf_block:

      layernorm_idx = 1
      mlp_layer_idx = 1

      if isinstance(outer_layer, SwinTransformerBlock3D):
          for inner_layer in outer_layer.layers:
        
              # Layer norm.
              if isinstance(inner_layer, tf.keras.layers.LayerNormalization):
                  layer_norm_prefix = (
                      f"{common_prefix}.{block_idx}.norm{layernorm_idx}"
                  )
                  inner_layer.gamma.assign(
                      tf.Variable(
                          np_state_dict[f"{layer_norm_prefix}.weight"]
                      )
                  )



                  inner_layer.beta.assign(
                      tf.Variable(np_state_dict[f"{layer_norm_prefix}.bias"])
                  )

                  layernorm_idx += 1

              # Window attention.
              elif isinstance(inner_layer, WindowAttention3D):
                  attn_prefix = f"{common_prefix}.{block_idx}.attn"

                  # Relative position.
                  inner_layer.relative_position_bias_table = (
                      modify_tf_block(
                          inner_layer.relative_position_bias_table,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_bias_table"
                          ] 
                      )
                  )
                  inner_layer.relative_position_index = (
                      modify_tf_block(
                          inner_layer.relative_position_index,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_index"
                          ]
                      )
                  )

                  # QKV.
                  inner_layer.qkv = modify_tf_block(
                      inner_layer.qkv,
                      np_state_dict[f"{attn_prefix}.qkv.weight"],
                      np_state_dict[f"{attn_prefix}.qkv.bias"]
                  )

                  # Projection.
                  inner_layer.proj = modify_tf_block(
                      inner_layer.proj,
                      np_state_dict[f"{attn_prefix}.proj.weight"],
                      np_state_dict[f"{attn_prefix}.proj.bias"]
                  )

              # MLP.
              elif isinstance(inner_layer, tf.keras.Model):
                  mlp_prefix = f"{common_prefix}.{block_idx}.mlp"
                  for mlp_layer in inner_layer.layers:
                      if isinstance(mlp_layer, tf.keras.layers.Dense):
                          mlp_layer = modify_tf_block(
                              mlp_layer,
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.weight"
                              ],
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                              ]
                          )
                          mlp_layer_idx += 1

          block_idx += 1
  return tf_block


### convert

In [ ]:
np_state_dict = pt_model.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}

tf_model.projection.layers[0] = modify_tf_block(tf_model.projection.layers[0]
        ,
        np_state_dict["patch_embed.proj.weight"],
        np_state_dict["patch_embed.proj.bias"])

tf_model.projection.layers[1] = modify_tf_block(
    tf_model.projection.layers[1],
    np_state_dict["patch_embed.norm.weight"],
    np_state_dict["patch_embed.norm.bias"])


layer_normalization_idx = -1

tf_model.layers[layer_normalization_idx] = modify_tf_block(
    tf_model.layers[layer_normalization_idx] ,
    np_state_dict["norm.weight"],
    np_state_dict["norm.bias"]
    )

# swin layers
for i in range(2, len(tf_model.layers) - 1):
    _ = modify_swin_blocks(np_state_dict,
                        f"layers.{i-2}",
                        tf_model.layers[i].layers)

In [ ]:
x_tf, x_pt = get_x(shape_of_input)

In [ ]:
layers_output_tf, y = tf_model(x_tf)
print("------")
layers_output_pt, z= pt_model(x_pt)

y.shape, z.shape


patch embed (10, 128, 16, 56, 56)
(10, 128, 50176)


2022-09-04 08:48:15.441420: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3147038720 exceeds 10% of free system memory.
2022-09-04 08:48:16.663478: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3147038720 exceeds 10% of free system memory.
2022-09-04 08:48:17.125025: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3147038720 exceeds 10% of free system memory.
2022-09-04 08:48:21.568714: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3147038720 exceeds 10% of free system memory.
2022-09-04 08:48:22.689181: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3147038720 exceeds 10% of free system memory.


------


(TensorShape([10, 1024, 16, 7, 7]), torch.Size([10, 1024, 16, 7, 7]))

In [ ]:
# comparing the outputs
np.testing.assert_allclose(y.numpy(), z.detach().numpy(), 1e-4, 1e-4)


In [ ]:
# Eplore the output of all layers
for layer in layers_output_pt:
    print("--------------",layer, "-------------\n TF: ", layers_output_tf[layer].numpy()[:1,:1,:1,:1,:10], "\n PT: ", layers_output_pt[layer].detach().numpy()[:1,:1,:1,:1,:10], "\n" )

-------------- PatchEmbed -------------
 TF:  [[[[[ 2.208807   -0.14340192  1.8464509   2.4065409  -0.80638236
     -1.7261122   0.93042624  0.959603   -3.3462863  -0.20938446]]]]] 
 PT:  [[[[[ 2.208807   -0.14340192  1.8464506   2.4065406  -0.8063823
     -1.7261125   0.93042624  0.9596029  -3.3462868  -0.20938458]]]]] 

-------------- drop_out -------------
 TF:  [[[[[ 2.208807   -0.14340192  1.8464509   2.4065409  -0.80638236
     -1.7261122   0.93042624  0.959603   -3.3462863  -0.20938446]]]]] 
 PT:  [[[[[ 2.208807   -0.14340192  1.8464506   2.4065406  -0.8063823
     -1.7261125   0.93042624  0.9596029  -3.3462868  -0.20938458]]]]] 

-------------- basic layer1 -------------
 TF:  [[[[[-0.02704892  0.02139371 -0.3663571  -0.2818588  -0.04923443
     -0.17632663 -0.05737334  0.28945643  0.44962877  0.06109022]]]]] 
 PT:  [[[[[-0.02704878  0.02139366 -0.36635745 -0.28185883 -0.04923458
     -0.17632675 -0.0573737   0.28945628  0.4496286   0.06109024]]]]] 

-------------- basic layer2

In [ ]:
# compare layers' output. It asserts  at basic layer1
for layer in layers_output_pt:

    abs_diff = np.sum( np.abs( layers_output_tf[layer].numpy() - layers_output_pt[layer].detach().numpy()))
    print(f"{abs_diff} Absolute Difference of  {layer}" )


2.9773874282836914 Absolute Difference of  PatchEmbed
2.9773874282836914 Absolute Difference of  drop_out
4.871058464050293 Absolute Difference of  basic layer1
3.4374818801879883 Absolute Difference of  basic layer2
21.05612564086914 Absolute Difference of  basic layer3
29.453521728515625 Absolute Difference of  basic layer4
4.482729911804199 Absolute Difference of  Final Output


In [ ]:
# compare layers' output. It asserts  at basic layer1
for layer in layers_output_pt:
    print("Testing", layer)
    np.testing.assert_allclose(layers_output_tf[layer].numpy(), layers_output_pt[layer].detach().numpy(), 1e-4, 1e-4)

Testing PatchEmbed
Testing drop_out
Testing basic layer1
Testing basic layer2
Testing basic layer3


AssertionError: 
Not equal to tolerance rtol=0.0001, atol=0.0001

Mismatched elements: 278 / 8028160 (0.00346%)
Max absolute difference: 0.01908875
Max relative difference: 9.957816
 x: array([[[[[ 1.517542e-01,  4.973968e-01,  4.700980e-01, ...,
            2.614313e-01,  4.739349e-01,  2.864167e-01],
          [ 8.704834e-01,  6.462526e-01,  9.973266e-01, ...,...
 y: array([[[[[ 1.517550e-01,  4.974024e-01,  4.700985e-01, ...,
            2.614331e-01,  4.739342e-01,  2.864182e-01],
          [ 8.704690e-01,  6.462545e-01,  9.973290e-01, ...,...

In [ ]:
a = a

In [ ]:
e = 1e-4
np.testing.assert_allclose(layers_output_tf["basic layer4"].numpy(), layers_output_pt["basic layer4"].detach().numpy(), e, e)


In [ ]:
# comparing the outputs
np.testing.assert_allclose(y.numpy(), z.detach().numpy(), 1e-4, 1e-4)


### Save and load tf model

### PT basic layer outputs comparison

In [ ]:
attempts = 2
# x_pt = torch.rand((1,3,8,224,224))

outputs = []

for i in range(attempts):
    layer_out , result = pt_model(x_pt)
    outputs.append(layer_out)

i = 0
for layer in outputs[0]:
    print("--------------", layer, "---------------")
    for idx, layer_out in enumerate(outputs) :
        print(f"attempt {idx} : ",layer_out[layer].detach().numpy()[:1,:1,:1,:1,:10], "\n")
    print()


### All close testing

Compare the first attempt with another attempt. Enter the attempt value in the following cell

In [ ]:
attempt_no = 1     # Change the attempt_no value to compare the first attempt with another attempt


In [ ]:
# PatchEmbed Layer
output1 = outputs[0]["PatchEmbed"]
output2 =  outputs[attempt_no]["PatchEmbed"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer1
output1 = outputs[0]["basic layer1"]
output2 =  outputs[attempt_no]["basic layer1"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer2

output1 = outputs[0]["basic layer2"]
output2 =  outputs[attempt_no]["basic layer2"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer3

output1 = outputs[0]["basic layer3"]
output2 =  outputs[attempt_no]["basic layer3"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer4

output1 = outputs[0]["basic layer4"]
output2 =  outputs[attempt_no]["basic layer4"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Final Output

output1 = outputs[0]["Final Output"]
output2 =  outputs[attempt_no]["Final Output"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)